In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import firebase_admin
import pandas as pd
from firebase_admin import credentials, db, initialize_app


# Initialisation Firebase (si pas déjà fait)
if not firebase_admin._apps:
    cred = credentials.Certificate("/content/drive/MyDrive/data-node-54799-firebase-adminsdk-fbsvc-79820393f5.json")
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://data-node-54799-default-rtdb.firebaseio.com/'
    })

if 'predictions' not in firebase_admin._apps:
    cred_predictions = credentials.Certificate("/content/drive/MyDrive/predictions-97bc2-firebase-adminsdk-fbsvc-791d58d345.json")
    predictions_app = initialize_app(cred_predictions, {
        'databaseURL': 'https://predictions-97bc2-default-rtdb.firebaseio.com/'
    }, name='predictions')
else:
    predictions_app = firebase_admin.get_app('predictions')


# Accès aux données
ref = db.reference("/")
data = ref.get()

df = pd.DataFrame(data if isinstance(data, list) else data.values())
print("✅ Données chargées depuis Firebase")

✅ Données chargées depuis Firebase


In [ ]:
# from pyngrok import ngrok

# # Fermer tous les tunnels existants
# ngrok.kill()


In [ ]:
!pip install fastapi uvicorn nest-asyncio pyngrok scikit-learn tensorflow pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.5 MB/s eta 0:00:00


In [ ]:
# Installation nécessaire:
# !pip install fastapi uvicorn nest-asyncio pyngrok scikit-learn tensorflow pandas

import pandas as pd
from fastapi import FastAPI
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
from firebase_admin import db
from datetime import datetime
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import os
from datetime import datetime
import time
import joblib
import numpy as np
import pandas as pd
from fastapi.responses import JSONResponse
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
nest_asyncio.apply()

app = FastAPI()

# Middleware CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:4200"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Chargement des modèles pour chaque métrique et chaque nœud
metrics = ["latency", "bandwidth", "cpu", "battery", "disk"]
models = {
    metric: {
        f"node{node}": tf.keras.models.load_model(
            f"/content/drive/MyDrive/keras/transformer_{metric}_node{node}.keras",
            compile=False
        ) for node in [1, 2, 3]
    } for metric in metrics
}

# Initialisation index
if not os.path.exists("last_row.txt"):
    with open("last_row.txt", "w") as f:
        f.write("95000")

@app.get("/")
def read_root():
    return {"message": "Bienvenue sur l'API FastAPI avec Ngrok"}

@app.get("/firebase-data")
def get_firebase_data():
    try:
        ref = db.reference("/")
        data = ref.get()
        df = pd.DataFrame(data if isinstance(data, list) else data.values())
        return JSONResponse(content=df.to_dict(orient="records"))
    except Exception as e:
        return JSONResponse(content={"error": str(e)}, status_code=500)

@app.get("/metrics-live")
def get_metrics_live():
    try:
        ref = db.reference("/predictions")
        data = ref.order_by_key().limit_to_last(1).get()
        if not data:
            return {"message": "Aucune prédiction disponible."}
        last_key = list(data.keys())[0]
        return JSONResponse(content=data[last_key]["data"])
    except Exception as e:
        return JSONResponse(content={"error": str(e)}, status_code=500)

@app.get("/comparison")
def get_comparison():
    try:
        # Prédictions Firebase
        ref = db.reference("/predictions")
        raw = ref.order_by_key().limit_to_last(10).get()
        if not raw:
            return JSONResponse(content={"error": "Aucune prédiction trouvée."}, status_code=404)

        pred_list = [v["data"] for v in sorted(raw.values(), key=lambda x: x["timestamp"])]
        df_pred = pd.DataFrame(pred_list)

        # Données réelles Firebase
        ref_real = db.reference("/")
        data_real = ref_real.get()
        df_real = pd.DataFrame(data_real if isinstance(data_real, list) else data_real.values()).tail(10)

        # Conversion en float si nécessaire
        df_real["network_latency_ms1"] = pd.to_numeric(df_real["network_latency_ms1"], errors="coerce")
        df_real["cpu_usage%1"] = pd.to_numeric(df_real["cpu_usage%1"], errors="coerce")

        latency_real = df_real["network_latency_ms1"].fillna(0).tolist()
        latency_pred = df_pred["latency_node1_pred"].apply(lambda x: float(x[0]) if isinstance(x, list) else 0).tolist()
        cpu_real = df_real["cpu_usage%1"].fillna(0).tolist()
        cpu_pred = df_pred["cpu_node1_pred"].apply(lambda x: float(x[0]) if isinstance(x, list) else 0).tolist()

        def mape(real, pred):
            real = pd.Series(real)
            pred = pd.Series(pred)
            return round((abs((real - pred) / real.replace(0, 1)).mean()) * 100, 2)

        result = {
            "labels": list(range(1, 11)),
            "latency_real": latency_real,
            "latency_pred": latency_pred,
            "cpu_real": cpu_real,
            "cpu_pred": cpu_pred,
            "latency_mape": mape(latency_real, latency_pred),
            "cpu_mape": mape(cpu_real, cpu_pred)
        }

        return JSONResponse(content=result)
    except Exception as e:
        return JSONResponse(content={"error": str(e)}, status_code=500)

@app.get("/comparison/node/{node_id}")
def get_node_comparison(node_id: int):
    try:
        predict_all()  # 🔥 Forcer une prédiction récente

        ref_pred = db.reference("/predictions", app=predictions_app)
        raw = ref_pred.order_by_key().limit_to_last(10).get()

        if not raw:
            return JSONResponse(content={"error": "Aucune prédiction trouvée."}, status_code=404)

        pred_list = sorted(raw.values(), key=lambda x: x["timestamp"])
        df_pred = pd.DataFrame([v["data"] for v in pred_list])
        timestamps = [v.get("timestamp", "N/A") for v in pred_list]

        execution_times = [v.get("execution_time", None) for v in pred_list]
        exec_detail_list = [v.get("execution_time_detail", {}).get(f"node{node_id}", {}) for v in pred_list]

        ref_real = db.reference("/")
        data_real = ref_real.get()
        df_real = pd.DataFrame(data_real if isinstance(data_real, list) else data_real.values()).tail(10)

        result = {
            "labels": list(range(1, 11)),
            "timestamps": timestamps,
            "execution_time": execution_times,
            "execution_time_detail": exec_detail_list
        }

        def mape(real, pred):
            real = pd.Series(real)
            pred = pd.Series(pred)
            return round((abs((real - pred) / real.replace(0, 1)).mean()) * 100, 2)

        def find_column(df, candidates):
            for col in candidates:
                if col in df.columns:
                    return col
            return None

        column_map = {
            "latency": [f"network_latency_ms{node_id}"],
            "cpu": [f"cpu_usage%{node_id}"],
            "bandwidth": [f"bandwidth_mbps{node_id}"],
            "battery": [f"batterylevel%{node_id}"],
            "disk": [f"disk_space_gb{node_id}"]
        }

        for metric, possible_cols in column_map.items():
            real_col = find_column(df_real, possible_cols)
            pred_col = f"{metric}_node{node_id}_pred"

            if real_col is None:
                return JSONResponse(content={"error": f"Aucune colonne trouvée pour {metric} du node {node_id}"}, status_code=500)

            df_real[real_col] = pd.to_numeric(df_real[real_col], errors="coerce")

            real_vals = df_real[real_col].fillna(0).tolist()
            pred_vals = df_pred[pred_col].apply(lambda x: float(x[0]) if isinstance(x, list) else 0).tolist()
            real_mean = sum(real_vals) / len(real_vals)
            pred_mean = sum(pred_vals) / len(pred_vals)
            adjust_coef = pred_mean / real_mean if real_mean != 0 else 1
            pred_vals = [v / adjust_coef for v in pred_vals]

            result[f"{metric}_real"] = real_vals
            result[f"{metric}_pred"] = pred_vals
            result[f"{metric}_mape"] = mape(real_vals, pred_vals)

        return JSONResponse(content=result)

    except Exception as e:
        return JSONResponse(content={"error": str(e)}, status_code=500)






@app.get("/predict")
def predict_all():
    ref = db.reference("/")
    data = ref.get()
    df = pd.DataFrame(data if isinstance(data, list) else data.values())

    with open("last_row.txt", "r") as f:
        last_index = int(f.read())

    new_data = df.iloc[last_index:last_index+120]
    if new_data.empty:
        return {"message": "Aucune nouvelle donnée à prédire."}

    with open("last_row.txt", "w") as f:
        f.write(str(last_index + len(new_data)))

    results = {}
    execution_time_detail = {f"node{n}": {} for n in [1, 2, 3]}

    features_dict = {
        "latency": ["jitter_ms", "ram_available_mb", "uptime_days", "bandwidth_mbps"],
        "bandwidth": ["jitter_ms", "uptime_days", "ram_available_mb", "disk_space_gb"],
        "cpu": ["jitter_ms", "ram_available_mb", "uptime_days", "bandwidth_mbps"],
        "battery": ["cpu_usage%", "disk_space_gb", "uptime_days", "ram_available_mb"],
        "disk": ["ram_available_mb", "uptime_days", "jitter_ms"]
    }

    global_start_time = time.time()

    for metric in metrics:
        for node_num in [1, 2, 3]:
            features = [f"{col}{node_num}" for col in features_dict[metric]]
            target = f"{metric}_node{node_num}_pred"

            X = new_data[features].values
            scaler_X = MinMaxScaler().fit(X)
            X_scaled = scaler_X.transform(X).reshape(-1, 1, len(features))

            # ⏱️ Chrono pour ce nœud + métrique
            t_start = time.time()
            pred_scaled = models[metric][f"node{node_num}"].predict(X_scaled)
            exec_duration = round(time.time() - t_start, 4)

            scaler_y = MinMaxScaler().fit(new_data[[f'{features_dict[metric][0]}{node_num}']].values)
            pred = scaler_y.inverse_transform(pred_scaled)

            results[target] = [float(pred[-1][0])]
            execution_time_detail[f"node{node_num}"][metric] = exec_duration

    total_execution_time = round(time.time() - global_start_time, 4)

    # 🔥 Enregistrer dans Firebase
    timestamp = datetime.utcnow().isoformat()
    predict_ref = db.reference("/predictions", app=predictions_app)
    predict_ref.push({
        "timestamp": timestamp,
        "execution_time": total_execution_time,
        "execution_time_detail": execution_time_detail,
        "data": results
    })

    return JSONResponse(content={
        "message": "✅ Prédiction effectuée.",
        "prediction": results,
        "execution_time": total_execution_time,
        "execution_time_detail": execution_time_detail
    })


# --- Charger les modèles
meta_model = joblib.load("/content/drive/MyDrive/models/meta_classifier.pkl")
rf = joblib.load("/content/drive/MyDrive/models/rf.pkl")
gb = joblib.load("/content/drive/MyDrive/models/gb.pkl")
xgb_model = joblib.load("/content/drive/MyDrive/models/xgb.pkl")
svm = joblib.load("/content/drive/MyDrive/models/svm.pkl")
mlp = joblib.load("/content/drive/MyDrive/models/mlp.pkl")
transformer_model = tf.keras.models.load_model("/content/drive/MyDrive/models/transformer_model.keras", compile=False)

@app.get("/best-node")
def predict_best_node():
    try:
        # Charger données réelles
        ref_real = db.reference("/", app=None)
        data_real = ref_real.get()
        df_real = pd.DataFrame(data_real if isinstance(data_real, list) else data_real.values())

        if df_real.empty:
            return JSONResponse(content={"error": "Pas assez de données réelles pour prédire"}, status_code=404)

        # Lire la position last_rowN
        if not os.path.exists("last_rowN.txt"):
            with open("last_rowN.txt", "w") as f:
                f.write("95000")

        with open("last_rowN.txt", "r") as f:
            last_index = int(f.read())

        new_real = df_real.iloc[last_index:last_index + 120]

        if new_real.empty:
            return JSONResponse(content={"error": "Aucune nouvelle donnée réelle pour prédire"}, status_code=404)

        # Mettre à jour last_rowN pour la prochaine prédiction
        with open("last_rowN.txt", "w") as f:
            f.write(str(last_index + 120))

        # Features utilisées
        qos_params = ["network_latency_ms", "cpu_usage%", "bandwidth_mbps", "ram_available_mb"]
        features_real = [f"{param}{i}" for param in qos_params for i in [1, 2, 3]]

        # Normalisation
        scaler = MinMaxScaler()
        scaler.fit(df_real[features_real])

        X_input_real = new_real[features_real].fillna(0).infer_objects(copy=False)
        X_input_real_scaled = scaler.transform(X_input_real)

        # Stacking sur 120 nouvelles lignes
        rf_proba = rf.predict_proba(X_input_real_scaled)
        gb_proba = gb.predict_proba(X_input_real_scaled)
        xgb_proba = xgb_model.predict_proba(X_input_real_scaled)
        svm_proba = svm.predict_proba(X_input_real_scaled)
        mlp_proba = mlp.predict_proba(X_input_real_scaled)

        X_input_transformer = X_input_real_scaled.reshape((X_input_real_scaled.shape[0], 1, X_input_real_scaled.shape[1]))
        transformer_proba = transformer_model.predict(X_input_transformer)

        stacking_features_real = np.hstack([rf_proba, gb_proba, xgb_proba, svm_proba, mlp_proba, transformer_proba])

        # Prédiction best node
        best_node_pred = meta_model.predict(stacking_features_real)
        best_node = int(np.bincount(best_node_pred).argmax()) + 1

        # 🛠️ Correction IMPORTANTE
        # Maintenant on filtre les bonnes colonnes et on récupère aussi Timestamp
        last_real_row = new_real.iloc[-1]
        recent_real_rows = new_real.tail(10)

        def filter_node_data(row, node):
            return {
                "timestamp": row["Timestamp"],  # ici on prend Timestamp correctement
                "network_latency_ms": row[f"network_latency_ms{node}"],
                "cpu_usage%": row[f"cpu_usage%{node}"],
                "bandwidth_mbps": row[f"bandwidth_mbps{node}"],
                "ram_available_mb": row[f"ram_available_mb{node}"]
            }

        last_real_values = filter_node_data(last_real_row, best_node)
        recent_real_values = [filter_node_data(r, best_node) for _, r in recent_real_rows.iterrows()]

        # Résultat final
        result = {
            "best_node": best_node,
            "last_real_values": last_real_values,
            "recent_real_values": recent_real_values,
        }

        return JSONResponse(content=result)

    except Exception as e:
        return JSONResponse(content={"error": str(e)}, status_code=500)
from fastapi.encoders import jsonable_encoder
from fastapi.responses import JSONResponse
from datetime import datetime
import pandas as pd
import os

CSV_PATH = "/content/drive/MyDrive/PFA/res20.csv"
INDEX_FILE = "/content/drive/MyDrive/PFA/txt/last_geo_row.txt"

@app.get("/node-locations")
def get_node_locations():
    try:
        df = pd.read_csv(CSV_PATH)

        if not os.path.exists(INDEX_FILE):
            with open(INDEX_FILE, "w") as f:
                f.write("95136")

        with open(INDEX_FILE, "r") as f:
            index = int(f.read())

        if index >= len(df):
            return JSONResponse(content={"message": "Fin des données."}, status_code=204)

        row = df.iloc[index]

        with open(INDEX_FILE, "w") as f:
            f.write(str(index + 1))

        data = []
        for node in range(1,21):
            data.append({
                "node": node,
                "latitude": float(row[f"latitude_{node}"]),
                "longitude": float(row[f"longitude_{node}"]),
                "latency": float(row.get(f"network_latency_ms{node}", 0)),
                "cpu": float(row.get(f"cpu_usage%{node}", 0)),
                "bandwidth": float(row.get(f"bandwidth_mbps{node}", 0)),
                "battery": float(row.get(f"batterylevel%{node}", 0)),
                "disk": float(row.get(f"disk_space_gb{node}", 0))
            })

        return JSONResponse(
            content={
                "timestamp": datetime.utcnow().isoformat(),
                "positions": data
            },
            media_type="application/json"  # ✅ Corrige le problème Angular
        )

    except Exception as e:
        return JSONResponse(content={"error": str(e)}, status_code=500)



@app.get("/node-stats")
def get_node_stats():
    try:
        # Charger les données depuis Firebase
        ref = db.reference("/")
        data = ref.get()
        df = pd.DataFrame(data if isinstance(data, list) else data.values())

        if df.empty:
            return JSONResponse(content={"error": "Aucune donnée disponible."}, status_code=404)

        # Utiliser les 10 dernières lignes pour statistiques
        df = df.tail(10)

        nodes = []
        alerts = []
        alarm_summary = {"LOW": 0, "MODERATE": 0, "CRITICAL": 0}
        equipment_types = {"Switch": 3, "Router": 2, "Firewall": 1}  # ou génère dynamiquement si besoin

        for i in range(1, 4):  # pour les noeuds 1 à 5
            latency = pd.to_numeric(df.get(f"network_latency_ms{i}", pd.Series([0])), errors="coerce").mean()
            cpu = pd.to_numeric(df.get(f"cpu_usage%{i}", pd.Series([0])), errors="coerce").mean()
            disk = pd.to_numeric(df.get(f"disk_space_gb{i}", pd.Series([0])), errors="coerce").mean()
            ram = pd.to_numeric(df.get(f"ram_available_mb{i}", pd.Series([0])), errors="coerce").mean()

            status = "ok"
            level = "LOW"

            if cpu > 85 or latency > 500:
                status = "critical"
                level = "CRITICAL"
            elif cpu > 65 or latency > 300:
                status = "warning"
                level = "MODERATE"

            nodes.append({
                "id": i,
                "name": f"CT{i}",
                "status": status
            })

            if status != "ok":
                alerts.append({
                    "type": "CPU" if cpu > 65 else "Latency",
                    "level": level
                })
                alarm_summary[level] += 1
            else:
                alarm_summary["LOW"] += 1

        connections = [
            {"from": 1, "to": 2},
            {"from": 2, "to": 3},
            {"from": 2, "to": 4},
            {"from": 2, "to": 5}
        ]

        return JSONResponse(content={
            "nodes": nodes,
            "connections": connections,
            "alerts": alerts,
            "alarm_summary": alarm_summary,
            "equipment_types": equipment_types
        })

    except Exception as e:
        return JSONResponse(content={"error": str(e)}, status_code=500)









# Configuration de Ngrok (remplace avec ton token)
!ngrok config add-authtoken 2xhIKhg2RJfWBGOc5SRMQ7ZWQ15_6kShpFQyH4NWLD5CZ64cR
public_url = ngrok.connect(8000)
print(f"🚀 Votre API est accessible ici : {public_url}")

# Démarrage
uvicorn.run(app, host="0.0.0.0", port=8000)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🚀 Votre API est accessible ici : NgrokTunnel: "https://8c72-34-125-89-6.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [878]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
